# [Fannie Mae Single-Family Loan Performance Data](https://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html)
[https://docs.rapids.ai/datasets/mortgage-data](https://docs.rapids.ai/datasets/mortgage-data)

In [ ]:
# Import Python modules from the Python Standard Library
import os
import glob
import collections

[https://docs.python.org/3/library/](https://docs.python.org/3/library/)

[https://automatetheboringstuff.com/](https://automatetheboringstuff.com/)

In [ ]:
# Import common Python modules for data science and machine learning applications
import matplotlib
import numpy
import pandas

[https://matplotlib.org/](https://matplotlib.org/)

[https://www.numpy.org/](https://www.numpy.org/)

[https://pandas.pydata.org/](https://pandas.pydata.org/)

In [ ]:
# Import NVIDIA's RAPIDS GPU-acclerated Python modules for data science and machine learning applications
import cudf

In [ ]:
# Define environment variables
USER = os.environ['USER']
#SLURM_JOB_ID = os.environ['SLURM_JOB_ID']
PROJECT_ID = 'use300'

In [ ]:
# Define paths to filesystems
#LOCAL_SCRATCH_DIR = os.path.join('/scratch', USER, SLURM_JOB_ID)
OASIS_SCRATCH_DIR = os.path.join('/oasis/scratch/comet', USER, 'temp_project')
OASIS_PROJECT_DIR = os.path.join('/oasis/projects/nsf', PROJECT_ID, USER)

In [ ]:
# Define path to root directory where the different datasets are located
FANNIE_MAE_ROOT_DIR = os.path.join(OASIS_SCRATCH_DIR,'data/fannie-mae/single-family-loan-performance')

In [ ]:
# List available datasets
sorted(os.listdir(FANNIE_MAE_ROOT_DIR))

In [ ]:
# Choose dataset
FANNIE_MAE_DATASET = 'mortgage_2000-2016_1gb'

In [ ]:
# Define path to dataset
FANNIE_MAE_DATASET_DIR = os.path.join(FANNIE_MAE_ROOT_DIR, FANNIE_MAE_DATASET)
FANNIE_MAE_DATASET_TARBALL = FANNIE_MAE_DATASET_DIR + '.tgz'

In [ ]:
# Define paths to loan acquisition and performance dataset directories
ACQUISITION_DATA_DIR = os.path.join(FANNIE_MAE_DATASET_DIR, 'acq')
PERFORMANCE_DATA_DIR = os.path.join(FANNIE_MAE_DATASET_DIR, 'perf')

In [ ]:
# List all files in loan acquisition data directory as well as size in bytes
acquisition_files = sorted(os.listdir(ACQUISITION_DATA_DIR))
for acq_file in acquisition_files:
    print(acq_file, os.stat(os.path.join(ACQUISITION_DATA_DIR, acq_file)).st_size)

In [ ]:
# List all files in loan performance data directory as well as size in bytes
performance_files = sorted(os.listdir(PERFORMANCE_DATA_DIR))
for perf_file in performance_files:
    print(perf_file, os.stat(os.path.join(PERFORMANCE_DATA_DIR, perf_file)).st_size)

In [ ]:
# Select the first year and quarter to begin analysis on
begin_year = 2000
begin_quarter = 1

# Select last year and quarter to end analysis on
end_year = 2016
end_quarter = 4

In [ ]:
# Create a list of all quarterly loan acquisition and performance data files required for analysis
acquisition_files = list()
performance_files = list()
years = list(range(begin_year, end_year+1))
for year in years:
    # Determine the number of quarters in each year to analyze
    if (year == begin_year):
        quarters = list(range(begin_quarter, 5))
    elif (year == end_year):
        quarters = list(range(1, end_quarter+1))
    else:
        quarters = list(range(1, 5))
    # Get absolute path for each quarterly loan acquisition and performance data file
    for quarter in quarters:
        acquisition_files.extend(glob.glob(os.path.abspath(os.path.join(ACQUISITION_DATA_DIR,'Acquisition_' + str(year) + 'Q' + str(quarter) + '.*'))))
        performance_files.extend(glob.glob(os.path.abspath(os.path.join(PERFORMANCE_DATA_DIR,'Performance_' + str(year) + 'Q' + str(quarter) + '.*'))))

In [ ]:
# Print first few lines from a loan acquisition file
number_of_lines = 5
with open(acquisition_files[0], 'r') as acq_file:
    for x in range(number_of_lines):
        print(acq_file.readline())

In [ ]:
# Print first few lines from a loan performance data file
number_of_lines = 5
with open(performance_files[0], 'r') as perf_file:
    for x in range(number_of_lines):
        print(perf_file.readline())

[https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_File_layout.pdf](https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_File_layout.pdf)

[https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_Glossary.pdf](https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_Glossary.pdf)

In [ ]:
# Define ordered dictionary of column data types for loan acquisition dataset
acquisition_data_columns = collections.OrderedDict()
acquisition_data_columns['loan_identifier'] = 'Int64'
acquisition_data_columns['origination_channel'] = 'category'
acquisition_data_columns['seller_name'] = 'category'
acquisition_data_columns['original_interest_rate'] = 'float64'
acquisition_data_columns['original_unpaid_principal_balance'] = 'Int64'
acquisition_data_columns['original_loan_term'] = 'Int64'
acquisition_data_columns['origination_date'] = 'object'
acquisition_data_columns['first_payment_date'] ='object'
acquisition_data_columns['original_loan_to_value'] = 'float64'
acquisition_data_columns['original_combined_loan_to_value'] = 'float64'
acquisition_data_columns['number_of_borrowers'] = 'Int64'
acquisition_data_columns['original_debt_to_income_ratio'] = 'float64'
acquisition_data_columns['borrower_credit_score_at_origination'] = 'Int64'
acquisition_data_columns['first_time_home_buyer_indicator'] = 'category'
acquisition_data_columns['loan_purpose'] = 'category'
acquisition_data_columns['property_type'] = 'category'
acquisition_data_columns['number_of_units'] = 'Int64'
acquisition_data_columns['occupancy_type'] = 'category'
acquisition_data_columns['property_state'] = 'category'
acquisition_data_columns['zip_code_short'] = 'Int64'
acquisition_data_columns['primary_mortgage_insurance_percent'] = 'float64'
acquisition_data_columns['product_type'] = 'category'
acquisition_data_columns['co-borrower_credit_score_at_origination'] = 'Int64'
acquisition_data_columns['mortgage_insurance_type'] = 'Int64'
acquisition_data_columns['relocation_mortgage_indicator'] = 'category'
acquisition_data_columns['year_quarter'] = 'Int64'

In [ ]:
# Define ordered dictionary of column data types for loan performance dataset
performance_data_columns = collections.OrderedDict()
performance_data_columns['loan_identifier'] = 'Int64'
performance_data_columns['monthly_reporting_period'] = 'object'
performance_data_columns['servicer_name'] = 'category'
performance_data_columns['current_interest_rate'] = 'float64'
performance_data_columns['current_actual_unpaid_principal_balance'] = 'float64'
performance_data_columns['loan_age'] = 'Int64'
performance_data_columns['remaining_months_to_legal_maturity'] = 'Int64'
performance_data_columns['adjusted_months_to_maturity'] = 'Int64'
performance_data_columns['maturity_date'] = 'object'
performance_data_columns['metropolitan_statistical_area'] = 'Int64'
performance_data_columns['current_loan_delinquency_status'] = 'Int64'
performance_data_columns['modification_flag'] = 'category'
performance_data_columns['zero_balance_code'] = 'category'
performance_data_columns['zero_balance_effective_date'] = 'object'
performance_data_columns['last_paid_installment_date'] = 'object'
performance_data_columns['foreclosure_date'] = 'object'
performance_data_columns['disposition_date'] = 'object'
performance_data_columns['foreclosure_costs'] = 'float64'
performance_data_columns['property_preservation_and_repair_costs'] = 'float64'
performance_data_columns['asset_recovery_costs'] = 'float64'
performance_data_columns['miscellaneous_holding_expenses_and_credits'] = 'float64'
performance_data_columns['associated_taxes_for_holding_property'] = 'float64'
performance_data_columns['net_sale_proceeds'] = 'float64'
performance_data_columns['credit_enhancement_proceeds'] = 'float64'
performance_data_columns['repurchase_make_whole_proceeds'] = 'float64'
performance_data_columns['other_forclosure_proceeds'] = 'float64'
performance_data_columns['non-interest_bearing_unpaid_principal_balance'] = 'float64'
performance_data_columns['principal_forgiveness_amount'] = 'float64'
performance_data_columns['repurchase_make_whole_proceeds_flag'] = 'category'
performance_data_columns['foreclosure_principal_write-off_amount'] = 'float64'
performance_data_columns['servicing_activity_indicator'] = 'category'

In [ ]:
%%time
# Read in all loan acquisition data into a Pandas DataFrame via pandas.read_csv method
#   https://tomaugspurger.github.io/modern-4-performance.html
acquisition_pdf = pandas.DataFrame()
for acq_file in acquisition_files:
    acquisition_pdf = acquisition_pdf.append(pandas.read_csv(acq_file, names=acquisition_data_columns.keys(), delimiter='|', dtype=acquisition_data_columns), ignore_index=True, sort=False)

In [ ]:
# Check loan acquisition datatypes
acquisition_pdf.dtypes

In [ ]:
# Convert some date-related object datatype columns in aquisition_pdf to datetime datatype columns
acquisition_pdf['origination_date'] = pandas.to_datetime(acquisition_pdf['origination_date'], format='%m/%Y')
acquisition_pdf['first_payment_date'] = pandas.to_datetime(acquisition_pdf['first_payment_date'], format='%m/%Y')

In [ ]:
# Convert some other object datatype columns in aquisition_pdf to category datatype columns
acquisition_pdf['seller_name'] = acquisition_pdf['seller_name'].astype('category')
acquisition_pdf['first_time_home_buyer_indicator'] = acquisition_pdf['first_time_home_buyer_indicator'].astype('category')
acquisition_pdf['loan_purpose'] = acquisition_pdf['loan_purpose'].astype('category')
acquisition_pdf['property_state'] = acquisition_pdf['property_state'].astype('category')

In [ ]:
# Re-check loan acquisition datatypes
acquisition_pdf.dtypes

In [ ]:
# Print first few rows of acquisition_pdf
acquisition_pdf.head()

In [ ]:
# Print last few rows of acquisition_pdf
acquisition_pdf.tail()

In [ ]:
# Print total number of loans in acquisition_pdf
len(acquisition_pdf)

In [ ]:
# Check the categories in the property_state column of acquisition_pdf
acquisition_pdf.property_state.cat.categories

In [ ]:
%%time
# Read in all loan performance data into a Pandas DataFrame via pandas.read_csv method
performance_pdf = pandas.DataFrame()
for perf_file in performance_files:
    performance_pdf = performance_pdf.append(pandas.read_csv(perf_file, names=performance_data_columns.keys(), delimiter='|', dtype=performance_data_columns), ignore_index=True, sort=False)

In [ ]:
# Check loan performance datatypes
performance_pdf.dtypes

In [ ]:
# Convert some date-related object datatype columns in performance_pdf to datetime datatype columns
performance_pdf['monthly_reporting_period'] = pandas.to_datetime(performance_pdf['monthly_reporting_period'], format='%m/%d/%Y')
performance_pdf['maturity_date'] = pandas.to_datetime(performance_pdf['maturity_date'], format='%m/%Y')
performance_pdf['zero_balance_effective_date'] = pandas.to_datetime(performance_pdf['zero_balance_effective_date'], format='%m/%Y')
performance_pdf['last_paid_installment_date'] = pandas.to_datetime(performance_pdf['last_paid_installment_date'], format='%m/%d/%Y')
performance_pdf['foreclosure_date'] = pandas.to_datetime(performance_pdf['foreclosure_date'], format='%m/%d/%Y')
performance_pdf['disposition_date'] = pandas.to_datetime(performance_pdf['disposition_date'], format='%m/%d/%Y')

In [ ]:
# Print first few rows of performance_pdf 
performance_pdf.head()

In [ ]:
# Print last few rows of acquisition_pdf
performance_pdf.tail()

In [ ]:
# Print total number of loans in acquisition_pdf
len(performance_pdf)

In [ ]:
# Check the categories in the modification_flag of performance_pdf
performance_pdf.modification_flag.cat.categories

[https://en.wikipedia.org/wiki/Conforming_loan](https://en.wikipedia.org/wiki/Conforming_loan)

In [ ]:
%%time 
# Filter the loan acquisition data using the pandas.DataFrame.query method ...
#
# How many loans were made for more than $400,000?
acquisition_pdf.query('original_unpaid_principal_balance > 400000')

In [ ]:
%%time
# How many loans were made in California for more than $400,000?
acquisition_pdf.query('property_state == "CA" and original_unpaid_principal_balance > 400000')

In [ ]:
%%time
# How many loans were made in either California or New York for more than $400,000?
acquisition_pdf.query('(property_state == "CA" or property_state == "NY") and original_unpaid_principal_balance > 400000')

In [ ]:
%%time
# How many loans were made in either California, New York, or Florida for more than $400,000 to principal borrowers who had a credit score of less than 650?
acquisition_pdf.query('(property_state == "CA" or property_state == "NY" or property_state =="FL") and original_unpaid_principal_balance > 400000 and borrower_credit_score_at_origination < 650')

In [ ]:
%%time
# Use pandas.DataFrame.groupby method to create summary statistics ...
#
# What was the total number of loans originated per month?
total_number_of_loan_acquisitions = acquisition_pdf.groupby('origination_date')['loan_identifier'].count().to_frame()
total_number_of_loan_acquisitions.plot(figsize=(16,8))
matplotlib.pyplot.show()

[https://fred.stlouisfed.org/series/FEDFUNDS](https://fred.stlouisfed.org/series/FEDFUNDS)

In [ ]:
%%time
# What was the total unpaid principal balance originated per month (in billions of dollars)?
total_unpaid_principal_balances  = acquisition_pdf.groupby('origination_date')['original_unpaid_principal_balance'].sum().to_frame().div(1000000000)
total_unpaid_principal_balances.plot(figsize=(16,8))
matplotlib.pyplot.show()

In [ ]:
%%time
# What was the average unpaid principal balance at origination?
average_unpaid_principal_balances  = acquisition_pdf.groupby('origination_date')['original_unpaid_principal_balance'].mean().to_frame()
average_unpaid_principal_balances.plot(figsize=(16,8))
matplotlib.pyplot.show()

In [ ]:
%%time
# What was the average loan to value ratio at originiation?
average_loan_to_value  = acquisition_pdf.groupby('origination_date')['original_loan_to_value'].mean().to_frame()
average_loan_to_value.plot(figsize=(16,8))
matplotlib.pyplot.show()

[https://en.wikipedia.org/wiki/Loan-to-value_ratio](https://en.wikipedia.org/wiki/Loan-to-value_ratio)

[https://fred.stlouisfed.org/series/ASPUS](https://fred.stlouisfed.org/series/ASPUS)

[https://en.wikipedia.org/wiki/Subprime_mortgage_crisis#/media/File:Subprime_mortgage_originations,_1996-2008.GIF](https://en.wikipedia.org/wiki/Subprime_mortgage_crisis#/media/File:Subprime_mortgage_originations,_1996-2008.GIF)

In [ ]:
%%time
# What was the average credit score of a borrower at originiation?
average_credit_score  = acquisition_pdf.groupby('origination_date')['borrower_credit_score_at_origination'].mean().to_frame()
average_credit_score.plot(figsize=(16,8))
matplotlib.pyplot.show()

In [ ]:
%%time
# What was the average debt-to-income ratio of a borrower at origination?
average_debt_to_income  = acquisition_pdf.groupby('origination_date')['original_debt_to_income_ratio'].mean().to_frame()
average_debt_to_income .plot(figsize=(16,8))
matplotlib.pyplot.show()

In [ ]:
%%time
# What was the average interest rate on a loan at origination?
average_interest_rate = acquisition_pdf.groupby('origination_date')['original_interest_rate'].mean().to_frame()
average_interest_rate.plot(figsize=(16,8))
matplotlib.pyplot.show()

[https://fred.stlouisfed.org/series/WALCL](https://fred.stlouisfed.org/series/WALCL)

In [ ]:
%%time
# Combine monthly summary statistics series into a single pandas DataFrame.
acquisition_summary_pdf = pandas.concat([total_number_of_loan_acquisitions, total_unpaid_principal_balances,average_unpaid_principal_balances,average_loan_to_value,average_credit_score,average_debt_to_income,average_interest_rate], axis=1)
acquisition_summary_pdf

In [ ]:
%%time
# Create a cuDF loan acquisition DataFrame on an NVIDIA GPU from the existing loan acquisition pandas DataFrame
acquisition_cudf = cudf.DataFrame.from_pandas(acquisition_pdf)

In [ ]:
# Re-check loan acquisition datatypes in acquisition_cudf
acquisition_cudf.dtypes

In [ ]:
# Print first few rows of acquisition_cudf
acquisition_cudf.head().to_pandas()

In [ ]:
# Print total number of loans in acquisition_cudf
len(acquisition_cudf)

In [ ]:
%%time
# Filter the loan acquisition data using the cuDF DataFrame.query method ...
# How many loans were made for more than $400,000?
print(acquisition_cudf.query('original_unpaid_principal_balance > 400000'))

In [ ]:
%%time
# What if we dump cuDF DataFrame from GPU to CPU?
acquisition_cudf.query('original_unpaid_principal_balance > 400000').to_pandas()

In [ ]:
%%time
# How many loans were made in California for more than $400,000?
acquisition_cudf.query('property_state == "CA" and original_unpaid_principal_balance > 400000')

In [ ]:
%%time
# cuDF does not (appear to) currently recognize string variables in query method ...
acquisition_cudf.query('property_state == 4 and original_unpaid_principal_balance > 400000').to_pandas()

In [ ]:
%%time
# How many loans were made in either California (4), New York (35), or Florida (9)for more than $400,000 to principal borrowers who had a credit score of less than 650?
acquisition_cudf.query('(property_state == 4 or property_state == 35 or property_state == 9) and original_unpaid_principal_balance > 400000 and borrower_credit_score_at_origination < 650').to_pandas()

In [ ]:
%%time
# Use cuDF.DataFrame.groupby method to create summary statistics ...
#
# What was the total number of loans originated per month?
total_number_of_loan_acquisitions_cudf = acquisition_cudf.groupby('origination_date')['loan_identifier'].count().to_frame()
total_number_of_loan_acquisitions_cudf.to_pandas().plot(figsize=(16,8))
matplotlib.pyplot.show()

In [ ]:
%%time
# Create a cuDF loan performance DataFrame on an NVIDIA GPU from the existing loan performance pandas DataFrame
# *** WARNING: The loan performance datasets are quite LARGE! You may run out of memory on a single GPU!
performance_cudf = cudf.DataFrame.from_pandas(performance_pdf)